#**Big Data Application in E-commense**
##——Customer Behavior analysis and recommendation
###Project Contributor : **Tao Liu**



In this project, we will use [Amazon Review Data](http://deepyeti.ucsd.edu/jianmo/amazon/index.html) and perform different kind of data anlysis methods to analyze the customer behaviors for buying good and then give recommendation based on analysis.

##**Step 0** - Package import
All the packages will be imported here.

In [29]:
import numpy
import sklearn
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import requests
import array

##**Step 1** - Data Implemtation and cleaning

We will start with Amazon Review Data implementation and cleaning them if possible.



In [30]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Gift_Cards.json.gz

--2020-11-09 18:17:52--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Gift_Cards.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 380174 (371K) [application/octet-stream]
Saving to: ‘meta_Gift_Cards.json.gz.5’

meta_Gift_Cards.jso 100%[===================>] 371.26K  1.30MB/s    in 0.3s    

2020-11-09 18:17:52 (1.30 MB/s) - ‘meta_Gift_Cards.json.gz.5’ saved [380174/380174]



In [50]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
      yield json.dumps(eval(l))
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
      df[i] = d
      i += 1
    return pd.DataFrame.from_dict(df, orient='index')
def build_database(asin_dict):
    f = open("output.strict", 'w')
    for l in parse("meta_Gift_Cards.json.gz"):
      (f.write(l + '\n'))
    df = getDF('meta_Gift_Cards.json.gz')[0]
    database =[]
    n = 2000
    for i in df:
      dictionary = json.loads(i)
      also_view = dictionary['also_view']
      also_view_reformat = []
      for i in also_view:
        try:
            also_view_reformat.append(asin_dict[i])
        except:
            also_view_reformat.append(n)
            asin_dict[i] = n
            n = n+1
      also_buy = dictionary['also_buy']
      also_buy_reformat = []
      for i in also_buy:
        try:
            also_buy_reformat.append(asin_dict[i])
        except:
            also_buy_reformat.append(n)
            asin_dict[i] = n
            n = n+1   
      similar_item = dictionary['similar_item']
      similar_item_reformat = []
      for i in similar_item:
        try:
             similar_item_reformat.append(asin_dict[i])
        except:
             similar_item_reformat.append(n)
             asin_dict[i] = n
             n = n+1     
      asin =dictionary['asin']
      asin = asin_dict[asin]
      sub_list =[asin, also_view_reformat,also_buy_reformat, similar_item_reformat]
      database.append(sub_list)
    return  database
def reformat_asin():
    f = open("output.strict", 'w')
    for l in parse("meta_Gift_Cards.json.gz"):
      (f.write(l + '\n'))
    df = getDF('meta_Gift_Cards.json.gz')[0]
    asin_dict = {}
    n = 1
    for i in df:
      dictionary = json.loads(i)
      asin = dictionary['asin']
      asin_dict[asin] = n
      n = n+1
    return asin_dict
asin_dict = reformat_asin()
print(asin_dict)    
database = build_database(asin_dict)
print("This is the data it contained in database",database[1])
print("This is the length of the asin",len(database))
print(len(database[1]))
print("This is the length of the asin_dict",len(asin_dict))

{'B001BKEWF2': 1, 'B001GXRQW0': 2, 'B001H53QE4': 3, 'B001H53QEO': 4, 'B001KMWN2K': 5, 'B001M1UVQO': 6, 'B001M1UVZA': 7, 'B001M5GKHE': 8, 'B002BSHDJK': 9, 'B002DN7XS4': 10, 'B002H9RN0C': 11, 'B002MS7BPA': 12, 'B002NZXF9S': 13, 'B002O018DM': 14, 'B002O0536U': 15, 'B002OOBESC': 16, 'B002PY04EG': 17, 'B002QFXC7U': 18, 'B002QTM0Y2': 19, 'B002QTPZUI': 20, 'B002SC9DRO': 21, 'B002UKLD7M': 22, 'B002VFYGC0': 23, 'B002VG4AR0': 24, 'B002VG4BRO': 25, 'B002W8YL6W': 26, 'B002XNLC04': 27, 'B002XNOVDE': 28, 'B002YEWXZ0': 29, 'B002YEWXMI': 30, 'B003755QI6': 31, 'B003CMYYGY': 32, 'B003NALDC8': 33, 'B003XNIBTS': 34, 'B003ZYIKDM': 35, 'B00414Y7Y6': 36, 'B0046IIHMK': 37, 'B004BVCHDC': 38, 'B004CG61UQ': 39, 'B004CZRZKW': 40, 'B004D01QJ2': 41, 'B004KNWWPE': 42, 'B004KNWWP4': 43, 'B004KNWWR2': 44, 'B004KNWWRC': 45, 'B004KNWWT0': 46, 'B004KNWWRW': 47, 'B004KNWWQ8': 48, 'B004KNWWNG': 49, 'B004KNWWPO': 50, 'B004KNWWXQ': 51, 'B004KNWWUE': 52, 'B004KNWWYU': 53, 'B004KNWWWC': 54, 'B004KNWX3A': 55, 'B004KNWX1W': 56, 


##**Step 2** - RandomForest Implementation


In [59]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
def RandomForest_classify():
        print(database['asin'])
        data_x = database[1,2,3]
        data_y = database[0]


        #preparing data for training
        X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.8, random_state=0)


        #training the algorithm
        clf = RandomForestClassifier(n_estimators = 10, n_jobs= 2, random_state = 0)
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        y_predict = clf.predict_proba(X_test)

        #evaluating the algorithm
        a=accuracy_score(y_test,y_pred)
        print("TIme is ", time.time()-first)
        return a
RandomForest_classify()


ValueError: ignored

##**Step 3** - Neural Network Implementation

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import keras
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import time

#reference: https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5

class NeuralNetworkClassifier:
    def classify(self,dataset):
            
            self.dataset = dataset#read the dataset

            #data_x is the attributes that the dataset has
            #data_y is the attributes that whether the recommendation is right or not
            data_x = dataset['similar_item','also_buy','also_view']
            data_y = dataset['asin']
            data_y = keras.utils.to_categorical(data_y, num_classes=None, dtype='float32')

            #preparing data for training 
            X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.8, random_state=0)

            #training the algorithm
            #creating model sequentially and the output of each layer we add is input to the next layer we specify
            model = Sequential()
            model.add(Dense(10, input_dim = 11, activation = 'relu'))
            model.add(Dense(5,activation='relu'))
            model.add(Dense(2, activation='softmax'))

            #specify the loss function and optimizer
            model.compile(loss='categorical_crossentropy', optimizer = 'adam',
                          metrics=['accuracy'])

            #training model
            history = model.fit(X_train, y_train, epochs = 10, batch_size = 64, verbose = 0)

            #check the accuracy
            y_pred = model.predict(X_test)

            pred = list()
            for i in range(len(y_pred)):
                pred.append(np.argmax(y_pred[i]))

                test = list()
            for i in range(len(y_test)):
                test.append(np.argmax(y_test[i]))

            a = accuracy_score(pred, test)
            return a





##**Step 4** - Recommendation Strategy Implementation

